In [1]:
import pandas as pd
import numpy as np
import regex as re
import math
import os

In [3]:
# Parse the template to extract: 
# - the instruction /prompt, 
# - the source
# - the target
# - the type (only target, or source and target)
# also returns a "normalized" version of the prompt
def parse_wmt_template(template):
    # print(template)
    if (template == "gpt3-fr-en" or template == "gpt3-en-fr"):
        template = template + "-target"
    template = re.sub("gpt\-3", "gpt3", template)
    template = re.sub("translate\_as\_", "translate_as-", template)
    template = re.sub("source\-target", "source+target", template)
    [prompt, src, trg, prptype] = template.split("-")
    
    template = re.sub("gpt3", "gpt-3", template)
    prompt   = re.sub("gpt3", "gpt-3", prompt)   
    return([prompt, src, trg, prptype, template])

print(parse_wmt_template("gpt-3-en-fr-source+target"))

['gpt-3', 'en', 'fr', 'source+target', 'gpt-3-en-fr-source+target']


In [4]:
# normalize post processing procedure, adding 'none' when none is used
def parse_wmt_postproc(postproc):
    p = re.sub('^[0-9]+\.?', '', str(postproc))
    if (p == "" or p == "nan" or p == "NaN"):
        p = "none"
    return(p)

# normalize model names
def parse_wmt_model(model):
    if (model == "bloom-6b3"):
        model = "bloom-7b1"
    return(model)

In [5]:
from pathlib import Path 
def collect_results(path=None, task = "*", metric = "bleu", shot="*"):
    """
    Collect results from all datasets as csv files
    """
    dfslst = []
    root_dir = Path(path)
    # files = source_dir.iterdir()
    pattern = task + "/" + shot + "/" + "*" + metric + "*.tsv"
    # print(pattern)
    files = root_dir.glob(pattern)
    # files = root_dir.glob('*/*/*bleu*.tsv')
    print(files)
    for file in files:
        # print(file.name)       
        dfslst.append(pd.read_csv(file, 
                                  sep="\t", 
                                  header=0, 
                                  dtype = {'model':str,'task':str,'template':str,'fewshot':int,'seed':int,'postproc':str,'timestamp':str,'filename':str,'BLEU':float},
                                  index_col=False))   
    allres = pd.concat(dfslst)
    post = allres['postproc'].apply(parse_wmt_postproc)
    allres['postproc'] = post
    prompt = allres['template'].apply(parse_wmt_template)
    # newdf = pd.DataFrame([c[0] for c in xa], [c[1] for c in xa], [c[2] for c in xa],[c[3] for c in xa])
    # newdf = pd.DataFrame({"name":[c[0] for c in xa],"src":[c[1] for c in xa], "trg":[c[2] for c in xa], "type":[c[3] for c in xa]})
    for (cname, cindex) in [("prompt", 0), ("src", 1), ("trg", 2), ("prptype", 3), ("template", 4)]:
        allres[cname] = [c[cindex] for c in prompt.values]
    return(allres)
    # df.assign(newdf)

rootdir  = "/Users/yvon/dat/Projects/2021-BigScience/mt-bigscience/outputs/"
allres   = collect_results(rootdir, task="wmt14*", metric="bleu")

# allcomet = collect_results(rootdir, task="wmt14*", metric="comet")
allres.to_csv("/Users/yvon/Downloads/allres.csv")

<generator object Path.glob at 0x7fc9b11143d0>


## Main table for the BLOOM paper

In [49]:
templateList = [
    "a_good_translation-fr-en-source+target", "a_good_translation-en-fr-source+target", \
    "gpt-3-en-fr-target", "gpt-3-fr-en-target", \
    "version-en-fr-target", "version-fr-en-target", \
    "xglm-en-fr-source+target", "xglm-fr-en-source+target", \
    "a_good_translation-en-hi-source+target", "a_good_translation-hi-en-source+target", 
    "gpt-3-en-hi-target", "gpt-3-hi-en-target", \
    "version-en-hi-target", "version-hi-en-target", \
    "xglm-en-hi-source+target", "xglm-hi-en-source+target"
    ] 

mask = pd.array(allres['fewshot'] < 2, dtype="boolean") & \
pd.array(allres['postproc'] == "none", dtype ="boolean") & \
pd.array(allres['model'] == "bloom", dtype="boolean") &\
pd.array(allres['template'].isin(templateList))
filtered = allres[mask]
selected = filtered.loc[:, ['model', 'fewshot', 'template', 'BLEU', 'src', 'trg',]]
selected.drop_duplicates(keep='first', inplace=True, ignore_index=True)
print(selected.sort_values(by=['template','fewshot', 'src', 'trg'], axis=0))

    model  fewshot                                template   BLEU src trg
16  bloom        0  a_good_translation-en-fr-source+target  15.38  en  fr
24  bloom        1  a_good_translation-en-fr-source+target  36.39  en  fr
0   bloom        0  a_good_translation-en-hi-source+target   1.90  en  hi
8   bloom        1  a_good_translation-en-hi-source+target  14.49  en  hi
17  bloom        0  a_good_translation-fr-en-source+target  14.15  fr  en
25  bloom        1  a_good_translation-fr-en-source+target  36.56  fr  en
1   bloom        0  a_good_translation-hi-en-source+target  10.19  hi  en
9   bloom        1  a_good_translation-hi-en-source+target  24.60  hi  en
18  bloom        0                      gpt-3-en-fr-target   7.90  en  fr
26  bloom        1                      gpt-3-en-fr-target  32.55  en  fr
2   bloom        0                      gpt-3-en-hi-target   0.26  en  hi
10  bloom        1                      gpt-3-en-hi-target   6.51  en  hi
19  bloom        0                    

## Main table for (Bawden and Yvon) ##
The first table is about getting all the results we have for one prompt (xglm source target), all languages pairs, in two versions. The models considered are:
- bloom
- t0
- opt
- T0pp

Caveat: currently we just compute and print the numbers, but do not produce the tex table

In [28]:
mask = pd.array(allres['prompt'] == "xglm",dtype="boolean") & \
    pd.array(allres['fewshot'] < 2, dtype="boolean") & \
    pd.array(allres['postproc'] != "newline-cut") &\
    (pd.array(allres['model'] == "bloom",dtype="boolean") |
     pd.array(allres['model'] == "t0",dtype="boolean") |
     pd.array(allres['model'] == "opt", dtype="boolean")) &\
    pd.array(allres['prptype'] == "source+target")
filtered = allres[mask]
selected = filtered.loc[:, ['model', 'fewshot', 'postproc', 'BLEU', 'src', 'trg',]]
selected.drop_duplicates(keep='first', inplace=True, ignore_index=True)
print(selected.sort_values(by=['fewshot', 'src', 'trg', 'model', 'postproc'], axis=0))

    model  fewshot                     postproc   BLEU src trg
24  bloom        0  newline-cut-custom-truncate  32.25  en  fr
25  bloom        0                         none  14.91  en  fr
28    opt        0  newline-cut-custom-truncate  18.86  en  fr
29    opt        0                         none  12.95  en  fr
32     t0        0  newline-cut-custom-truncate   1.21  en  fr
33     t0        0                         none   1.21  en  fr
0   bloom        0  newline-cut-custom-truncate  12.10  en  hi
1   bloom        0                         none   6.80  en  hi
4     opt        0  newline-cut-custom-truncate   0.11  en  hi
5     opt        0                         none   0.14  en  hi
8      t0        0  newline-cut-custom-truncate   0.16  en  hi
9      t0        0                         none   0.16  en  hi
26  bloom        0  newline-cut-custom-truncate  37.16  fr  en
27  bloom        0                         none  15.52  fr  en
30    opt        0  newline-cut-custom-truncate  33.18 

----
### Analysis of languages occurring in the output translations ###
This analysis relies on fast text, and is based on the same configurations as for the previous table: 
one prompt, all language pairs
----

In [69]:
import fasttext
lid_model = fasttext.load_model('/Users/yvon/CNRS-Cloud/Projects/2021-BigScience/Analysis/Outputs/lid.176.ftz')

def detector(text):
    text = str(text)
    # return empty string if there is no tweet
    if text.isspace():
        return ""
    else:
        # get first item of the prediction tuple, then split by "__label__" and return only language code
        text = re.sub("\n", " ", text)
        return lid_model.predict(text)[0][0].split("__label__")[1]

def perlang_analysis(model, postproc):
    mask = pd.array(allres['prompt'] == "xglm", dtype="boolean") & \
    pd.array(allres['fewshot'] < 10, dtype="boolean") & \
    pd.array(allres['postproc'] == postproc) &\
    pd.array(allres['model'] == model) &\
    pd.array(allres['prptype'] == "source+target")
    filtered = allres[mask]
    selected = filtered.loc[:, ['task', 'fewshot', 'filename', 'src', 'trg']]
    # tasks = selected.loc[:,'task'].drop_duplicates().to_list()
    # print(tasks)
    selected['path'] = rootdir + selected['task'] + "/" + selected['fewshot'].astype(str) + "-shot/tsv/" + selected['filename']
    selected['langpair'] = selected['src'] + "-" + selected['trg']
    pairs =  selected.loc[:,'langpair'].drop_duplicates().to_list()
    print(pairs)
    
    for pair in pairs:
        mask = pd.array(selected['langpair'] == pair)
        filtered = selected[mask]
        
        L = filtered.loc[:,'path'].to_list()
        S = filtered.loc[:,'fewshot'].to_list()
        L.sort()
        S.sort()
        allstats = []
        
        for wmtoutput in L:
    
            # print('\n**** wmt output:', wmtoutput, '\n')
            # Make sure quoting = 3 - this matters to avoid errors
            translations = pd.read_csv(wmtoutput, sep='\t', quoting=3, engine = "python", names = ['ctx','ref','hyp'], dtype={0:str, 1:str, 2:str})
            # print(translations.shape)
            # print(translations.head())
    
            translations['empty'] = translations.apply(lambda row: len(str(row.hyp)) == 0, axis=1)
            translations['ldiff'] = translations.apply(lambda row: len(str(row.ref)) - len(str(row.hyp)),axis=1)
            translations['lang'] = translations.apply(lambda row: detector(row.hyp), axis=1)
            bylang = translations.groupby('lang').describe()
            empty = translations.groupby('empty').describe()
             
            allstats.append(bylang)
            # print(empty)
        alltab = pd.concat(allstats, axis = 1, keys = S)
        
        if (alltab.shape[0] > 0):
            cols = [(key, 'ldiff', heading) for (key, heading) in [(k, h) for k in S for h in ['count','mean']]]
            print(alltab.to_latex(
                    # header = ['N', 'Delta','N', 'Delta','N', 'Delta','N', 'Delta'],
                    columns = cols,
                    multicolumn = True,
                    float_format = "%.1f",
                    caption = "Output language identification results for " + pair + " (" + postprocs + ")"
                    ))
        
        
models = "bloom"
postprocs = "newline-cut-custom-truncate"
perlang_analysis(models, postprocs)

['en-hi', 'hi-en', 'en-fr', 'fr-en']
\begin{table}
\centering
\caption{Output language identification results for en-hi (newline-cut-custom-truncate)}
\begin{tabular}{lrrrrrrrr}
\toprule
{} & \multicolumn{2}{l}{0} & \multicolumn{2}{l}{1} & \multicolumn{2}{l}{2} & \multicolumn{2}{l}{5} \\
{} & \multicolumn{2}{l}{ldiff} & \multicolumn{2}{l}{ldiff} & \multicolumn{2}{l}{ldiff} & \multicolumn{2}{l}{ldiff} \\
{} &  count &   mean &  count &  mean &  count &  mean &  count &  mean \\
\midrule
ceb &    1.0 & -150.0 &    NaN &   NaN &    NaN &   NaN &    NaN &   NaN \\
en  &  476.0 &   10.5 &   48.0 &  12.4 &   71.0 &  13.9 &   26.0 &  18.8 \\
eo  &    1.0 & -134.0 &    NaN &   NaN &    NaN &   NaN &    NaN &   NaN \\
fi  &    1.0 &   19.0 &    NaN &   NaN &    NaN &   NaN &    NaN &   NaN \\
fr  &    2.0 &   94.5 &    NaN &   NaN &    NaN &   NaN &    NaN &   NaN \\
gom &    2.0 &    6.5 &    1.0 &   4.0 &    NaN &   NaN &    1.0 &   0.0 \\
hi  & 1998.0 &    9.3 & 2431.0 &   6.0 & 2403.0 &   5

### Third table, Number of shots ###
bloom xglm prompt, all directions and pairs, show the effect of increasing the fewshot = 0-5


In [336]:
def pershot_analysis():
    mask = pd.array(allres['prompt'] == "xglm", dtype="boolean") & \
    pd.array(allres['fewshot'] < 6 , dtype="boolean") & \
    pd.array(allres['postproc'] != "newline-cut") &\
    pd.array(allres['model'] == "bloom", dtype="boolean") &\
    pd.array(allres['prptype'] == "source+target")
    filtered = allres[mask]
    selected = filtered.loc[:, ['model', 'fewshot', 'postproc', 'spBLEU', 'src', 'trg',]]
    selected.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    print(selected.sort_values(by=['model', 'src', 'trg', 'postproc', 'fewshot'], axis=0))

pershot_analysis()

    model  fewshot                     postproc  spBLEU src trg
16  bloom        0  newline-cut-custom-truncate   32.25  en  fr
20  bloom        1  newline-cut-custom-truncate   36.29  en  fr
28  bloom        2  newline-cut-custom-truncate   37.62  en  fr
24  bloom        5  newline-cut-custom-truncate   37.88  en  fr
17  bloom        0                         none   14.91  en  fr
21  bloom        1                         none   27.83  en  fr
29  bloom        2                         none   35.09  en  fr
25  bloom        5                         none   37.89  en  fr
0   bloom        0  newline-cut-custom-truncate   12.10  en  hi
4   bloom        1  newline-cut-custom-truncate   15.73  en  hi
12  bloom        2  newline-cut-custom-truncate   15.78  en  hi
8   bloom        5  newline-cut-custom-truncate   16.46  en  hi
1   bloom        0                         none    6.80  en  hi
5   bloom        1                         none   13.62  en  hi
13  bloom        2                      

### Show variability by prompt ### 
This analysis is run for all models, with a restricted list of prompts.

In [432]:
def permodel_analysis(allres, metric="BLEU", metriclab = "BLEU"):
    modelList = [
    "bloom-560m", 
    "bloom", 
    "bloom-1b1", 
#    "bloom-1b7", 
    "bloom-3b", 
    "bloom-7b1"
    ]
    templateList = [
    "a_good_translation-en-fr-source+target", "a_good_translation-en-fr-target", \
    "a_good_translation-fr-en-source+target", "a_good_translation-fr-en-target", \
    "gpt-3-en-fr-target", "gpt-3-fr-en-target", \
    "translate_as-en-fr-target", "translate_as-fr-en-target", \
    "version-en-fr-target", "version-fr-en-target", \
    "xglm-en-fr-source+target", "xglm-en-fr-target", \
    "xglm-fr-en-source+target", "xglm-fr-en-target",
    "a_good_translation-en-hi-source+target", "a_good_translation-en-hi-target", \
    "a_good_translation-en-hi-target", "a_good_translation-hi-en-target", \
    "gpt-3-en-hi-target", "gpt-3-hi-en-target", \
    "translate_as-en-hi-target", "translate_as-hi-en-target", \
    "version-en-hi-target", "version-hi-en-target", \
    "xglm-en-hi-source+target", "xglm-en-hi-target", \
    "xglm-hi-en-source+target", "xglm-hi-en-target",
    ] 

    mask = pd.array(allres['fewshot'] < 2 , dtype="boolean") & \
    pd.array(allres['postproc'] != "newline-cut") & \
    pd.array(allres['model'].isin(modelList)) & \
    pd.array(allres['template'].isin(templateList))

    filtered = allres[mask]
    selected = filtered.loc[:, ['model', 'fewshot', 'postproc', metric, 'src', 'trg', 'prompt', 'prptype']]
    selected.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    alltab = []

    hien = [('en', 'hi'), ('hi', 'en')]
    fren = [('en', 'fr'), ('fr', 'en')]
    for postproc in ["none", "newline-cut-custom-truncate"]:
        for langpairs in hien, fren:
            alllst = []
            keylst = []
            for shot in [0,1]:
                for (src, trg) in langpairs:    
                # print(sel.head())
                    mask = pd.array(selected['src'] == src, dtype="boolean") & pd.array(selected['trg'] == trg, dtype="boolean")
                    sel = selected[mask]
                    sel = sel[sel['fewshot'] == shot] # .loc[:,['model_name','score']]
                    sel = sel[sel['postproc'] == postproc] # .loc[:,['model_name','score']]
                    filt = sel.loc[:,['model','BLEU']]
                    # print(sel.head())
                    # print("\n**** Source / target", src, trg, " **** shots", shot, "**** postproc", postproc, "****")
                    tab = filt.groupby(['model']).describe(percentiles=[])
                    alllst.append(tab)
                    keylst.append(src + "-" + trg + " (" + str(shot) + "-shot)")
                    # 
                    if (tab.shape[0] > 10):
                        print(tab.to_latex(
                        #header = ['avg', 'min' , 'max'],
                        columns = [(metric, 'mean'), (metric, 'min'), (metric, 'max')],
                        float_format = "%.2f",
                        caption = metriclab + " by model for " + src + "-" + trg + " (" + str(shot) + " shot, " + postproc + ")"
                        ))
            alltab = pd.concat(alllst, axis = 1, keys = keylst)
            # print(alltab)
            print("\n**** Source / target", src, trg, "**** postproc", postproc, " ****")
            if (alltab.shape[0] > 0):
                cols = [(key, metric, heading) for (key, heading) in [(k, h) for k in keylst for h in ['mean','min', 'max']]]
                print(alltab.to_latex(
                        #header = ['avg', 'min' , 'max'],
                        columns = cols,
                        multicolumn = True,
                        float_format = "%.2f",
                        caption = metriclab + " by model for " + src + "-" + trg + " (" + postproc + ")"
                        ))
    return(1)
                      
permodel_analysis(allres, metric="BLEU", metriclab = "BLEU scores") 


**** Source / target hi en **** postproc none  ****
\begin{table}
\centering
\caption{BLEU scores by model for hi-en (none)}
\begin{tabular}{lrrrrrrrrrrrr}
\toprule
{} & \multicolumn{3}{l}{en-hi (0-shot)} & \multicolumn{3}{l}{hi-en (0-shot)} & \multicolumn{3}{l}{en-hi (1-shot)} & \multicolumn{3}{l}{hi-en (1-shot)} \\
{} & \multicolumn{3}{l}{spBLEU} & \multicolumn{3}{l}{spBLEU} & \multicolumn{3}{l}{spBLEU} & \multicolumn{3}{l}{spBLEU} \\
{} &           mean &  min &  max &           mean &  min &   max &           mean &  min &   max &           mean &  min &   max \\
model      &                &      &      &                &      &       &                &      &       &                &      &       \\
\midrule
bloom      &           2.05 & 0.26 & 6.80 &           8.59 & 0.66 & 13.04 &          12.88 & 6.51 & 14.55 &          20.58 & 9.98 & 25.80 \\
bloom-1b1  &           0.07 & 0.02 & 0.11 &           1.40 & 0.00 &  4.54 &           1.44 & 0.05 &  3.08 &           5.18 & 0.00 &  8

/var/folders/0r/l7kgc3mn5vl8vkcw3kjk41gh0000gp/T/ipykernel_14651/1649084562.py:69: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(alltab.to_latex(
/var/folders/0r/l7kgc3mn5vl8vkcw3kjk41gh0000gp/T/ipykernel_14651/1649084562.py:69: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(alltab.to_latex(
/var/folders/0r/l7kgc3mn5vl8vkcw3kjk41gh0000gp/T/ipykernel_14651/1649084562.py:69: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base impl

1

In [433]:
# Like previous function, could be improved by dropping one level of indexing
def perprompt_analysis(allres, metric="BLEU", metriclab = "BLEU"):
    modelList = [
    "bloom-560m", 
    "bloom", 
    "bloom-1b1", 
#    "bloom-1b7", 
    "bloom-3b", 
    "bloom-7b1"
    ]
    templateList = [
    "a_good_translation-en-fr-source+target", "a_good_translation-en-fr-target", \
    "a_good_translation-fr-en-source+target", "a_good_translation-fr-en-target", \
    "gpt-3-en-fr-target", "gpt-3-fr-en-target", \
    "translate_as-en-fr-target", "translate_as-fr-en-target", \
    "version-en-fr-target", "version-fr-en-target", \
    "xglm-en-fr-source+target", "xglm-en-fr-target", \
    "xglm-fr-en-source+target", "xglm-fr-en-target",
    "a_good_translation-en-hi-source+target", "a_good_translation-en-hi-target", \
    "a_good_translation-hi-en-source+target", "a_good_translation-hi-en-target", \
    "gpt-3-en-hi-target", "gpt-3-hi-en-target", \
    "translate_as-en-hi-target", "translate_as-hi-en-target", \
    "version-en-hi-target", "version-hi-en-target", \
    "xglm-en-hi-source+target", "xglm-en-hi-target", \
    "xglm-hi-en-source+target", "xglm-hi-en-target",
    ] 

    mask = pd.array(allres['fewshot'] < 2 , dtype="boolean") & \
    pd.array(allres['postproc'] != "newline-cut") & \
    pd.array(allres['model'].isin(modelList)) & \
    pd.array(allres['template'].isin(templateList))

    filtered = allres[mask]
    selected = filtered.loc[:, ['model', 'fewshot', 'postproc', 'BLEU', 'src', 'trg', 'prompt', 'prptype']]
    selected.drop_duplicates(keep='first', inplace=True, ignore_index=True)

    hien = [('en', 'hi'), ('hi', 'en')]
    fren = [('en', 'fr'), ('fr', 'en')]
    for postproc in ["none", "newline-cut-custom-truncate"]:
        for langpairs in hien, fren:
            alllst = []
            keylst = []
            for shot in [0,1]:
                for (src, trg) in langpairs:    
                # print(sel.head())
                    mask = pd.array(selected['src'] == src, dtype="boolean") & pd.array(selected['trg'] == trg, dtype="boolean")
                    sel = selected[mask]
                    sel = sel[sel['fewshot'] == shot] # .loc[:,['model_name','score']]
                    sel = sel[sel['postproc'] == postproc] # .loc[:,['model_name','score']]
                    filt = sel.loc[:,['prompt', 'prptype', 'BLEU']]
                    # print(sel.head())
                    # print("\n**** Source / target", src, trg, " **** shots", shot, "**** postproc", postproc, "****")
                    tab = filt.groupby(['prompt', 'prptype']).describe(percentiles=[])
                    alllst.append(tab)
                    keylst.append(src + "-" + trg + " (" + str(shot) + "-shot)")
                    # print(tab)
                    #print(tab.to_latex(
                    # header = ['avg', 'min' , 'max'],
                    #    columns = [(metric, 'mean'), (metric, 'min'), (metric, 'max')],
                    #   float_format = "%.2f",
                    #   caption = metriclab + " by prompt for " + src + "-" + trg + " (" + str(shot) + " shot, " + postproc + ")"
                    #))
            alltab = pd.concat(alllst, axis = 1, keys = keylst)
            # print(alltab)
            print("\n**** Source / target", src, trg, "**** postproc", postproc, " ****")
            if (alltab.shape[0] > 0):
                cols = [(key, metric, heading) for (key, heading) in [(k, h) for k in keylst for h in ['mean','min', 'max']]]
                print(alltab.to_latex(
                        #header = ['avg', 'min' , 'max'],
                        columns = cols,
                        multicolumn = True,
                        float_format = "%.2f",
                        caption = metriclab + " per prompt for " + src + "-" + trg + " (" + postproc + ")"
                        ))
    return(1)

perprompt_analysis(allres, metric="BLEU", metriclab = "BLEU scores") 


**** Source / target hi en **** postproc none  ****
\begin{table}
\centering
\caption{BLEU scores per prompt for hi-en (none)}
\begin{tabular}{llrrrrrrrrrrrr}
\toprule
     &        & \multicolumn{3}{l}{en-hi (0-shot)} & \multicolumn{3}{l}{hi-en (0-shot)} & \multicolumn{3}{l}{en-hi (1-shot)} & \multicolumn{3}{l}{hi-en (1-shot)} \\
     &        & \multicolumn{3}{l}{spBLEU} & \multicolumn{3}{l}{spBLEU} & \multicolumn{3}{l}{spBLEU} & \multicolumn{3}{l}{spBLEU} \\
     &        &           mean &  min &  max &           mean &  min &   max &           mean &  min &   max &           mean &  min &   max \\
prompt & prptype &                &      &      &                &      &       &                &      &       &                &      &       \\
\midrule
a\_good\_translation & source+target &           0.72 & 0.07 & 1.90 &           4.84 & 0.88 & 10.19 &           5.80 & 0.34 & 14.49 &          13.06 & 2.78 & 24.60 \\
     & target &           0.22 & 0.06 & 0.77 &           6.29 & 1.

/var/folders/0r/l7kgc3mn5vl8vkcw3kjk41gh0000gp/T/ipykernel_14651/870374572.py:68: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(alltab.to_latex(
/var/folders/0r/l7kgc3mn5vl8vkcw3kjk41gh0000gp/T/ipykernel_14651/870374572.py:68: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(alltab.to_latex(
/var/folders/0r/l7kgc3mn5vl8vkcw3kjk41gh0000gp/T/ipykernel_14651/870374572.py:68: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base impleme

1